# Net Liquidity

Net Liquidity is:  

Fed Balance Sheet - Treasury General Account - Reverse Repo = Net Liquidity  

Fed Balance Sheet: https://fred.stlouisfed.org/series/WALCL  
Treasury General Account: https://fred.stlouisfed.org/series/WTREGEN  
Reverse Repo: https://fred.stlouisfed.org/series/WLRRAL

In [23]:
import pandas as pd
import yfinance as yf

fed_df = pd.read_csv('./Inputs/FRED_Net_Liquidity/WALCL.csv')
fed_df['DATE'] = pd.to_datetime(fed_df['DATE'])
display(fed_df.head(2))
tga_df = pd.read_csv('./Inputs/FRED_Net_Liquidity/WTREGEN.csv')
tga_df['DATE'] = pd.to_datetime(tga_df['DATE'])
display(tga_df.head(2))
rev_df = pd.read_csv('./Inputs/FRED_Net_Liquidity/WLRRAL.csv')
rev_df['DATE'] = pd.to_datetime(rev_df['DATE'])
display(rev_df.head(2))

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "BTC-USD SPY",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

# display(data.tail(5))

btc_df = data['BTC-USD'][['Close']]
btc_df = btc_df.rename(columns={'Close':'BTC_price'})
display(btc_df.tail(2))

sp500_df = data['SPY'][['Close']]
sp500_df = sp500_df.rename(columns={'Close':'SP500_price'})
display(sp500_df.tail(2))

,DATE,WALCL
0,2002-12-18,719542.0
1,2002-12-25,732059.0


,DATE,WTREGEN
0,1986-01-08,4.140
1,1986-01-15,5.032


,DATE,WLRRAL
0,2002-12-18,21905.0
1,2002-12-25,20396.0


[*********************100%***********************]  2 of 2 completed


,BTC_price
Date,
2022-08-27,20041.738281
2022-08-29,19674.894531


,SP500_price
Date,
2022-08-27,NaN
2022-08-29,NaN


In [24]:
combined_df.index

Index(['2002-12-18', '2002-12-25', '2003-01-01', '2003-01-08', '2003-01-15',
       '2003-01-22', '2003-01-29', '2003-02-05', '2003-02-12', '2003-02-19',
       ...
       '2022-06-22', '2022-06-29', '2022-07-06', '2022-07-13', '2022-07-20',
       '2022-07-27', '2022-08-03', '2022-08-10', '2022-08-17', '2022-08-24'],
      dtype='object', name='Date', length=1028)

In [25]:
type(combined_df.index[0])

str

In [26]:
type(btc_df.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [29]:
combined_df = pd.merge(fed_df,tga_df,on='DATE',how='left')
combined_df = pd.merge(combined_df,rev_df,on='DATE',how='left')
combined_df = combined_df.rename(columns={'DATE':'Date','WALCL':'FedBalanceSheet','WTREGEN':'TGA','WLRRAL':'RevRepos'})
combined_df['NetLiquidity'] = combined_df['FedBalanceSheet'] - combined_df['TGA'] - combined_df['RevRepos']
combined_df = combined_df.set_index('Date')
combined_df = pd.merge(combined_df,btc_df,left_index=True,right_index=True)
combined_df = pd.merge(combined_df,sp500_df,left_index=True,right_index=True)
combined_df

,FedBalanceSheet,TGA,RevRepos,NetLiquidity,BTC_price,SP500_price
Date,,,,,,
2002-12-18,719542.0,5.959,21905.0,697631.041,NaN,61.414032
2003-01-08,723762.0,4.829,18709.0,705048.171,NaN,62.808575
2003-01-15,720074.0,5.244,17813.0,702255.756,NaN,63.502712
2003-01-22,735953.0,6.133,18523.0,717423.867,NaN,60.595604
2003-01-29,712809.0,7.162,18466.0,694335.838,NaN,59.434143
...,...,...,...,...,...,...
2022-07-27,8890004.0,602.945,2484253.0,6405148.055,22930.548828,401.040009
2022-08-03,8874620.0,594.115,2447840.0,6426185.885,22846.507812,414.450012
2022-08-10,8879138.0,557.265,2445699.0,6432881.735,23947.642578,419.989990


In [32]:
for col in combined_df.columns:
    combined_df[f'{col}_z'] = (combined_df[f'{col}'] - combined_df[f'{col}'].mean()) / combined_df[f'{col}'].std()

combined_df.tail(5)

,FedBalanceSheet,TGA,RevRepos,NetLiquidity,BTC_price,SP500_price,FedBalanceSheet_z,TGA_z,RevRepos_z,NetLiquidity_z,BTC_price_z,SP500_price_z
Date,,,,,,,,,,,,
2022-07-27,8890004.0,602.945,2484253.0,6405148.055,22930.548828,401.040009,2.473716,1.124406,5.018297,1.724739,0.622627,2.208378
2022-08-03,8874620.0,594.115,2447840.0,6426185.885,22846.507812,414.450012,2.466901,1.099388,4.936357,1.735559,0.617546,2.336628
2022-08-10,8879138.0,557.265,2445699.0,6432881.735,23947.642578,419.989990,2.468902,0.994977,4.931539,1.739003,0.684118,2.389611
2022-08-17,8849762.0,545.321,2467848.0,6381368.679,23335.998047,426.649994,2.455888,0.961135,4.981381,1.712509,0.647140,2.453305
2022-08-24,8851436.0,535.267,2494360.0,6356540.733,21395.019531,413.670013,2.456630,0.932649,5.041041,1.699739,0.529793,2.329168


In [34]:
import hvplot.pandas

net_liquidiyt_df = combined_df[['NetLiquidity_z','BTC_price_z','SP500_price_z']]
net_liquidity_plot = net_liquidiyt_df.hvplot.line(title='Net Liquidity v BTC Price v SP500 Price',
                                   grid=True)
net_liquidity_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)